In [2]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from matplotlib import cm

In [3]:
def my_logpdf(x, u, covar):
    """This function calculates the value of a multivariate Gaussian pdf at a given point x, with mean u and covariance covar

    Args:
        x : The point at which the pdf is evaluated. Nx1 column vector.
        u : The mean vector of the Gaussian distribution. Nx1 column vector.
        covar : The covariance matrix of the Gaussian distribution. NxN matrix.

    Returns:
        The value of the Gaussian pdf at x.
    """
    k = len(x)  # dimension
    a = np.transpose(x - u)
    b = np.linalg.inv(covar)
    c = x - u
    d = np.matmul(a, b)
    e = np.matmul(d, c)
    numer = np.exp(-0.5 * e)
    f = (2 * np.pi)**k
    g = np.linalg.det(covar)
    denom = np.sqrt(f * g)
    pdf = numer / denom
    return pdf



In [4]:
base_obstacle = lambda x,y, obstacle: 20*my_logpdf(np.array([x,y]),obstacle,np.diag(np.array([0.02, 0.02], dtype=np.float32)))
base_goal = lambda X,Y, goal: 30*((X-goal[0])**2 + (Y-goal[1])**2)
dropwave = lambda X,Y,f,s,p,o,k,A: A*(o + np.cos(f*np.sqrt((X/k)**2+(Y/k)**2))/((s)*(X**2+Y**2)+p))
dropwave_attenuation = lambda X,Y,f,s,p,o,k,A,goal,obstacle,alpha,beta: dropwave(X,Y,f,s,p,o,k,A) * np.exp(alpha*np.sqrt((X-goal[0])**2+(Y-goal[1])**2)) * np.exp(-beta*np.sqrt((X-obstacle[0])**2+(Y-obstacle[1])**2))

In [6]:
f = 1
s = 1
p = 1
o = 1
k = 1
A = 1
zita = 1
alpha = 1
beta = 1
goal = np.array([0,-1.0])
obstacle = np.array([0,0])

In [7]:
%matplotlib tk
n_points_x = 300
n_points_y = 200
x = np.linspace(-1.5,1.5,n_points_x)
y = np.linspace(-1,1,n_points_y)

X, Y = np.meshgrid(x,y)

base_obstacle_map= np.zeros((n_points_x,n_points_y))

for i in range(n_points_x):
    for j in range(n_points_y):
        base_obstacle_map[i,j] = base_obstacle(x[i],y[j], obstacle)

In [8]:
cost = dropwave_attenuation(X,Y,f,s,p,o,k,A,goal, obstacle, alpha , beta) + base_obstacle_map.T + base_goal(X,Y,goal)
cost_gradient = np.gradient(cost)
cost_gradient_module = np.sqrt(cost_gradient[0]**2 + cost_gradient[1]**2)

fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(2,1,1,projection='3d')

#p: Inversamente proprozionale al picco
#s: Inversamente proprozionale al cratere
#f: legato alla frequenza
#o: Un offset
surf = ax.plot_surface(X,Y,cost, cmap=cm.coolwarm)
# fix axis to robotarium size
ax.set_xlim(-1.5,1.5)
ax.set_ylim(-1,1)
ax.set_zlim(-10,100)

ax2 = fig.add_subplot(2,1,2)
heathmap = ax2.pcolormesh(X,Y,cost_gradient_module, vmin=0, vmax=2)
fig.colorbar(heathmap, ax=ax2)

In [16]:
import ipywidgets as widgets
from IPython.display import display

# Define the sliders
f_slider = widgets.FloatSlider(min=0.00001, max=250.0, step=0.00001, value=100, description='f:')
s_slider = widgets.FloatSlider(min=0.0001, max=1000.0, step=0.001, value=100, description='s:')
p_slider = widgets.FloatSlider(min=0.0001, max=1500.0, step=0.001, value=30.0, description='p:')
o_slider = widgets.FloatSlider(min=0.0, max=10.0, step=0.001, value=0.0, description='o:')
k_slider = widgets.FloatSlider(min=0.0, max=100.0, step=0.001, value=7.41, description='k:')
A_slider = widgets.FloatSlider(min=0.0, max=1000.0, step=0.001, value=37.16, description='A:')
alpha_slider = widgets.FloatSlider(min=-2.0, max=2.0, step=0.00001, value=1.0, description='alpha:')
beta_slider = widgets.FloatSlider(min=0, max=10.0, step=0.00001, value=1.0, description='beta:')

# Display the sliders
display(f_slider, s_slider, p_slider, o_slider, k_slider, A_slider, alpha_slider, beta_slider)

# Update the function to use the slider values
def update_plot():
    f = f_slider.value
    s = s_slider.value
    p = p_slider.value
    o = o_slider.value
    k = k_slider.value
    A = A_slider.value
    alpha = alpha_slider.value
    beta = beta_slider.value
    
    cost = dropwave_attenuation(X,Y,f,s,p,o,k,A,goal, obstacle, alpha , beta) + base_obstacle_map.T + base_goal(X,Y,goal)
    cost_gradient = np.gradient(cost)
    cost_gradient_module = np.sqrt(cost_gradient[0]**2 + cost_gradient[1]**2)
    
    
    ax.clear()
    surf = ax.plot_surface(X,Y,cost, cmap=cm.coolwarm)
    ax.set_xlim(-1.5,1.5)
    ax.set_ylim(-1,1)
    ax.set_zlim(-5,100)
    
    ax2.clear()
    heathmap = ax2.pcolormesh(X,Y,cost_gradient_module, vmin=0, vmax=2)


# Call the update function when a slider value changes
f_slider.observe(lambda change: update_plot(), names='value')
s_slider.observe(lambda change: update_plot(), names='value')
p_slider.observe(lambda change: update_plot(), names='value')
o_slider.observe(lambda change: update_plot(), names='value')
k_slider.observe(lambda change: update_plot(), names='value')
A_slider.observe(lambda change: update_plot(), names='value')
alpha_slider.observe(lambda change: update_plot(), names='value')
beta_slider.observe(lambda change: update_plot(), names='value')

# Initial plot
update_plot()

FloatSlider(value=100.0, description='f:', max=250.0, min=1e-05, step=1e-05)

FloatSlider(value=100.0, description='s:', max=1000.0, min=0.0001, step=0.001)

FloatSlider(value=30.0, description='p:', max=1500.0, min=0.0001, step=0.001)

FloatSlider(value=0.0, description='o:', max=10.0, step=0.001)

FloatSlider(value=7.41, description='k:', step=0.001)

FloatSlider(value=37.16, description='A:', max=1000.0, step=0.001)

FloatSlider(value=1.0, description='alpha:', max=2.0, min=-2.0, step=1e-05)

FloatSlider(value=1.0, description='beta:', max=10.0, step=1e-05)